# 多智能体对话和相声

欢迎来到正式的代码实战环节。接下来我们需要先安装一些必要的环境和基础的配置。

首先，使用conda新建一个虚拟的python环境并激活进入

```bash
conda create -n autogen python=3.11
conda activate autogen
```

接下来，安装本课程学习中所必要的依赖

```bash
cd <current-directory>
pip install -r requirements.txt
```

安装完毕，让我们马上开始基于AutoGen框架的Agent设计范式学习吧~</br>
在这一节课中，我们会实现两个智能体合作讲相声的一个有趣例子😊

## 基础配置

In [1]:
from utils import get_openai_api_key, get_openai_base_url
OPENAI_API_KEY = get_openai_api_key()
OPENAI_BASE_URL = get_openai_base_url()
llm_config = {"model": "gpt-3.5-turbo"}

## 定义一个AutoGen智能体

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here you go: Why did the scarecrow win an award? Because he was outstanding in his field!


In [5]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Please share the joke you'd like me to repeat.


由此可见，我们执行了两次`generate_reply`函数，但是智能体的第二次回复并没有继承上一次的交互记忆。

## 对话

让Cathy和Joe两个Agent建立对话，它们的对话内容作为记忆可以被保存下来

In [6]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**注意**： 您每次执行都可能会得到不同的笑话，因为这和大语言模型的底层逻辑，预测下一个生成token的最大概率有关。

In [7]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let's get this comedy train chugging along. So, I recently went on a strict diet and it taught me one important lesson - never grocery shop when you're hungry. I ended up with a cart full of snacks that were either half-eaten or had "family size" written on them... and by family, I mean just me. But hey, at least my pantry is prepared for any sudden snack attacks, right?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's so relatable! I mean, who knew family size meant just one hungry person, right? Speaking of snacking, I recently tried to work out at home to stay in shape, but those fitness videos are no joke. I paused the video to take a break, and it buffered long enough for me to order a pizza, eat it, and then hit play again - and the instruct

## 打印一些结果

你可以在控制台打印出下列信息：

1. 对话的历史记录
2. 对话的花费
3. 对话的总结

In [8]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe! Alright, let's get this comedy train chugging along. "
             'So, I recently went on a strict diet and it taught me one '
             "important lesson - never grocery shop when you're hungry. I "
             'ended up with a cart full of snacks that were either half-eaten '
             'or had "family size" written on them... and by family, I mean '
             'just me. But hey, at least my pantry is prepared for any sudden '
             'snack attacks, right?',
  'role': 'user'},
 {'content': "Haha, that's so relatable! I mean, who knew family size meant "
             'just one hungry person, right? Speaking of snacking, I recently '
             'tried to work out at home to stay in shape, but those fitness '
             'videos are no joke. I paused the video to take a break, and it '
             'buffered long enough for me to order a pizza, eat it, and then

In [9]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 277,
                                                             'cost': 0.0006180000000000001,
                                                             'prompt_tokens': 405,
                                                             'total_tokens': 682},
                                      'total_cost': 0.0006180000000000001},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 277,
                                                             'cost': 0.0006180000000000001,
                                                             'prompt_tokens': 405,
                                                             'total_tokens': 682},
                                      'total_cost': 0.0006180000000000001}}


In [11]:
pprint.pprint(chat_result.summary)

("Haha, that's impressive dedication right there, Joe! It's like workout "
 'buffering is the universe\'s way of saying, "Are you sure about this whole '
 'fitness thing?" I feel like I should make a workout video where the warm-up '
 'is just trying to find the motivation to press play. The real exercise is '
 'fighting the urge to switch over to Netflix instead. But hey, at least you '
 'got some quality pizza time in, right? Priorities, my friend, priorities.')


这里可以看到对话的总结完全是cathy对joe说的内容，也就是最后一轮对话的结尾。这是因为默认的总结方式就是取最后的一段历史对话记录。

## 对对话内容进行一个更好的总结

In [12]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let's get this comedy train chugging along. So, I recently went on a strict diet and it taught me one important lesson - never grocery shop when you're hungry. I ended up with a cart full of snacks that were either half-eaten or had "family size" written on them... and by family, I mean just me. But hey, at least my pantry is prepared for any sudden snack attacks, right?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's so relatable! I mean, who knew family size meant just one hungry person, right? Speaking of snacking, I recently tried to work out at home to stay in shape, but those fitness videos are no joke. I paused the video to take a break, and it buffered long enough for me to order a pizza, eat it, and then hit play again - and the instruct

In [14]:
pprint.pprint(chat_result.summary)

('Watch out for the "family size" label when shopping hungry and be prepared '
 'for long workout video buffering breaks with a pizza pitstop. Priorities, my '
 'friend, priorities.')


我们在`initiate_chat`函数中新加入了两个参数，`summary_method`和`summary_prompt`，这将改变原先默认的总结方法。现在我们使用大语言模型来帮助我们整理回顾并总结对话的历史信息，这将会得到一个更符合我们理解的一个结果。

## 对话终止

对话可以被一个结束条件所终止

In [15]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [16]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Have you heard about the claustrophobic astronaut? He just needed a little space!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one, Joe! How do you organize a space party? You planet!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, great one, Cathy! You're on fire with the space jokes. Did you hear about the mathematician who's afraid of negative numbers? He will stop at nothing to avoid them!

--------------------------------------------------------------------------------
cathy (to j

这里我们将终止条件设置为了当对话内容表达式中出现`I gotta go`这一字符串时终止对话。在此之前我们的两位Agent进行了多轮的交互。

In [17]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was about the guy who invented Lifesavers and how he made a mint.

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go.

--------------------------------------------------------------------------------


这里我们可以再次从先前结束的那轮对话开始继续让两个Agent进行交互。可以看到由于有对话记忆，能够正确的输出最后一轮的笑话。此外，在cathy输出`I gotta go`之后对话再次满足停止条件并终止。